In [13]:
# Fizzbuzz

for i in range(1, 15+1):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


In [3]:
from markdown import markdown

def convert_to_markdown(text):
  """Converts plain text to Markdown format.

  Args:
      text: The plain text string to be converted.

  Returns:
      A string containing the converted Markdown text.
  """

  # Optional: You can add additional extensions here
  # extensions = ['extra']  # For tables, footnotes, etc.

  return markdown(text)

# Example usage
text = "This is some plain text. \nHere are some **bold** and *italic* words. \nWe can also add headings: ### This is a level 2 heading"
markdown_text = convert_to_markdown(text)

print(markdown_text)


<p>This is some plain text. 
Here are some <strong>bold</strong> and <em>italic</em> words. 
We can also add headings: ### This is a level 2 heading</p>


In [29]:
import pandas as pd
import sqlite3


file = 'test_data.csv'
conn = sqlite3.connect(':memory:')
df = pd.read_csv(file)
df.to_sql('data', conn, index=False, if_exists='replace')

10155

In [32]:
columns_info = df.dtypes.to_dict()
formatted_metadata = "\n".join([f"{col}: {dtype}" for col, dtype in columns_info.items()])

In [40]:
import os
import dotenv
from groq import Groq

dotenv.load_dotenv()
api_key_llama = os.getenv("GROQ_API_KEY")
if not api_key_llama:
    raise ValueError("GROQ_API_KEY environment variable not set")

def get_llama_assistance(prompt, formatted_metadata, table_name):
    main_purpose = f"""
    As an SQL Query Expert, your primary role is to understand the given data, answer the questions based on the provided input and generate accurate SQL queries ONLY. 
    Remember, you only have to answer the Query for the given input in a single line, don't give any explanation, just the query. Don't use [\n] please. 
    Here are the column names with respect to their information: 
    Here are the column names with respect to their information: 
    {formatted_metadata}
    The table name is {table_name}
    Here is/are the Questions:"""

    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "user",
                "content": f"{main_purpose} {prompt}"
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        temperature=1.4,
        max_tokens=8192,
        top_p=1,
        stream=True,
        stop=None,
    )

    response_text = ""
    for chunk in completion:
        response_text += chunk.choices[0].delta.content or ""
    
    # Remove backticks from the response
    cleaned_response = response_text.replace("```", "").strip()
    
    return cleaned_response

In [ ]:
df.name = "file"

In [38]:
query = "List the occupations where the average medical insurance is less than the overall average medical insurance."
sql_query = get_llama_assistance(query, formatted_metadata, table_name)
sql_query

'SELECT occupation, AVG(medical_insurance) FROM datatable GROUP BY occupation HAVING AVG(medical_insurance) < (SELECT AVG(medical_insurance) FROM datatable);'

In [39]:
result_df = pd.read_sql_query(sql_query, conn)

DatabaseError: Execution failed on sql 'SELECT occupation, AVG(medical_insurance) FROM datatable GROUP BY occupation HAVING AVG(medical_insurance) < (SELECT AVG(medical_insurance) FROM datatable);': no such table: datatable

In [28]:
import sqlite3
import json
from django.http import JsonResponse

def get_query(request):
    if request.method == 'POST':
        file = request.FILES.get('file')
        query = request.POST.get('query')
        
        if file and query:
            # Create a temporary SQLite database
            conn = sqlite3.connect(':memory:')
            df = pd.read_csv(file)
            df.to_sql('data', conn, index=False, if_exists='replace')
            
            columns_info = df.dtypes.to_dict()
            formatted_metadata = "\n".join([f"{col}: {dtype}" for col, dtype in columns_info.items()])
            
            # Get the SQL query from LLM
            sql_query = get_llama_assistance(query, formatted_metadata)
            
            try:
                # Execute the SQL query
                result_df = pd.read_sql_query(sql_query, conn)
                conn.close()
                
                # Convert the result to JSON
                result_json = result_df.to_json(orient='records')
                
                return JsonResponse({'result': json.loads(result_json)})
            except Exception as e:
                conn.close()
                return JsonResponse({'error': str(e)}, status=400)
    
    return JsonResponse({'error': 'Invalid request'}, status=400)